# Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/Demo/quick_start.ipynb)

In [ ]:
!pip install indox
!pip install openai
!pip install chromadb

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Initial Setup

The following imports are essential for setting up the Indox application. These imports include the main Indox retrieval augmentation module, question-answering models, embeddings, and data loader splitter.

In [2]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


### Generating response using OpenAI's language models 
OpenAIQA class is used to handle question-answering task using OpenAI's language models. This instance creates OpenAiEmbedding class to specifying embedding model. Here ChromaVectorStore handles the storage and retrieval of vector embeddings by specifying a collection name and sets up a vector store where text embeddings can be stored and queried.

In [3]:
from indox.llms import OpenAi
from indox.embeddings import OpenAiEmbedding
openai_qa = OpenAi(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")
embed_openai = OpenAiEmbedding(api_key=OPENAI_API_KEY,model="text-embedding-3-small")

from indox.vector_stores import Chroma
db = Chroma(collection_name="sample",embedding_function=embed_openai)
indox.connect_to_vectorstore(vectorstore_database=db)

INFO: Initializing OpenAi with model: gpt-3.5-turbo-0125
INFO: OpenAi initialized successfully
INFO: Initialized OpenAI embeddings with model: text-embedding-3-small
INFO: Connection to the vector store database established successfully


### load and preprocess data
This part of code demonstrates how to load and preprocess text data from a file, split it into chunks, and store these chunks in the vector store that was set up previously.

In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

In [5]:
file_path = "sample.txt"

In [6]:
from indox.data_loader_splitter import UnstructuredLoadAndSplit
loader_splitter = UnstructuredLoadAndSplit(file_path=file_path,max_chunk_size=400)
docs = loader_splitter.load_and_chunk()

INFO: UnstructuredLoadAndSplit initialized successfully
INFO: Getting all documents
INFO: Starting processing
INFO: Using title-based chunking
INFO: Completed chunking process
INFO: Successfully obtained all documents


In [7]:
indox.store_in_vectorstore(docs=docs)

INFO: Storing documents in the vector store
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


### Retrieve relevant information and generate an answer
The main purpose of these lines is to perform a query on the vector store to retrieve the most relevant information (top_k=5) and generate an answer using the language model.

In [8]:
query = "How Cinderella reach her happy ending?"
retriever = indox.QuestionAnswer(vector_database=db, llm=openai_qa, top_k=5)

invoke(query) method sends the query to the retriever, which searches the vector store for relevant text chunks and uses the language model to generate a response based on the retrieved information.
Context property retrieves the context or the detailed information that the retriever used to generate the answer to the query. It provides insight into how the query was answered by showing the relevant text chunks and any additional information used.

In [9]:
answer = retriever.invoke(query)
context = retriever.context

INFO: Retrieving context and scores from the vector database
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Generating response
INFO: Response generated successfully
INFO: Query answered successfully


In [11]:
answer

"Cinderella reached her happy ending by being kind, patient, and having a pure heart. Despite facing mistreatment from her step-family, she remained humble and continued to do good deeds. With the help of a little white bird, she was able to receive what she wished for and planted a branch from a hazel bush on her mother's grave, which grew into a beautiful tree. Eventually, during a festival where the prince was to choose a bride, Cinderella's pure heart and beauty shone through, and with the help of the two white doves, she was able to prove her identity as the true bride. This led to her marrying the prince and living happily ever after."

### With AgenticRag

AgenticRag stands for Agentic Retrieval-Augmented Generation. This concept combines retrieval-based methods and generation-based methods in natural language processing (NLP). The key idea is to enhance the generative capabilities of a language model by incorporating relevant information retrieved from a database or a vector store. 
 AgenticRag is designed to provide more contextually rich and accurate responses by utilizing external knowledge sources. It retrieves relevant pieces of information (chunks) from a vector store based on a query and then uses a language model to generate a comprehensive response that incorporates this retrieved information.

In [ ]:
agent = indox.AgenticRag(llm=openai_qa,vector_database=db,top_k=5)
agent.run(query)

## Evaluation

In [27]:
from indox.evaluation import Evaluation
evaluator = Evaluation(["BertScore", "Toxicity",  "Reliability", "Fairness" , "Readibility"])

2024-07-02 10:27:22,324 INFO:Use pytorch device: cpu


In [28]:
inputs = {
    "question" : query,
    "answer" : answer,
    "context" : context
}
result = evaluator(inputs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
result

,0
Precision,0.493488
Recall,0.517235
F1-score,0.505083
Toxicity,0.081307
hallucination_score,0.620000
Fairness,0.496180
Perplexity,38.828209
ARI,12.800000
Flesch-Kincaid Grade Level,10.100000
